**Doc2Vec Tutorial **

In [ ]:
import re 
import pandas as pd
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import matplotlib.pyplot as plt
import nltk
nltk.download('all')
!python -m spacy download en_core_web_sm
import spacy

2022-03-31 20:20:49,491 : INFO : 'pattern' package not found; tag filters are not available for English


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

In [ ]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Mounted at /gdrive
/gdrive


Data Processing

In [ ]:
df = pd.read_csv('/gdrive/MyDrive/Group 4_Distributional Semantics_EmilyYeacker_JessicaCarter/Doc2Vec Bible/BasicEnglishBible.csv')
print(df.shape)
df.tail()

(31103, 5)


,id,b,c,v,t
31098,66022017,66,22,17,"And the Spirit and the bride say, Come. And le..."
31099,66022018,66,22,18,For I say to every man to whose ears have come...
31100,66022019,66,22,19,And if any man takes away from the words of th...
31101,66022020,66,22,20,"He who gives witness to these things says, Tru..."
31102,66022021,66,22,21,The grace of the Lord Jesus be with the saints...


In [ ]:
df['t'] = df['t'].apply(str)
data = df.t.values.tolist()
data_words = list(data)
print(list(data_words[:1]))

['At the first God made the heaven and the earth.']


In [ ]:
#Import Stopwords
from nltk.corpus import stopwords
stop_words = stopwords.words ('english')
stop_words.extend(['from', 'to', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', 'many', 'na', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need', 'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come', 'br', 'sub', 'mech','non'])

In [ ]:
df['t'] = df['t'].apply(str)
data = df.t.values.tolist()
data_words = list(data)
print(list(data_words[:1]))

['At the first God made the heaven and the earth.']


In [ ]:
# Compute bigrams.
import gensim
from gensim.models import Phrases
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

2022-03-31 20:22:10,737 : INFO : collecting all words and their counts
2022-03-31 20:22:10,738 : INFO : PROGRESS: at sentence #0, processed 0 words and 0 word types
2022-03-31 20:22:14,263 : INFO : PROGRESS: at sentence #10000, processed 1410045 words and 1031 word types
2022-03-31 20:22:18,677 : INFO : PROGRESS: at sentence #20000, processed 2669377 words and 1159 word types
2022-03-31 20:22:23,100 : INFO : PROGRESS: at sentence #30000, processed 3991122 words and 1229 word types
2022-03-31 20:22:23,702 : INFO : collected 1258 word types from a corpus of 4146400 words (unigram + bigrams) and 31103 sentences
2022-03-31 20:22:23,713 : INFO : using 1258 counts as vocab in Phrases<0 vocab, min_count=5, threshold=100, max_vocab_size=40000000>
/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")
2022

In [ ]:
# Add n-grams and lemmatization
def process_words(texts, stop_words=stop_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """Remove Stopwords, Form Bigrams, Trigrams and Lemmatization"""
    texts = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
    texts = [bigram_mod[doc] for doc in texts]
    texts = [trigram_mod[bigram_mod[doc]] for doc in texts]
    texts_out = []
    nlp = spacy.load('en_core_web_sm')
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    # remove stopwords once more after lemmatization
    texts_out = [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts_out]    
    return texts_out

In [ ]:
data_ready = process_words(data_words)

In [ ]:
#Split the data into train and test
from gensim import corpora
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.model_selection import train_test_split

train, test = train_test_split(data_ready, test_size=.30, random_state=43)

In [ ]:
import gensim 
def create_tagged_document(list_of_list_of_words):
    for i, list_of_words in enumerate(list_of_list_of_words):
        yield gensim.models.doc2vec.TaggedDocument(list_of_words, [i])
train_data = list(create_tagged_document(train))
test_data = list(create_tagged_document(test))

In [ ]:
print(train_data[:5])
print(test_data[:5])

[TaggedDocument(words=['force', 'wall', 'town', 'responsible', 'land', 'town'], tags=[0]), TaggedDocument(words=['let', 'mind', 'sport'], tags=[1]), TaggedDocument(words=['brother', 'comfort', 'faith'], tags=[2]), TaggedDocument(words=['give', 'worship', 'faith', 'news', 'news', 'preacher'], tags=[3]), TaggedDocument(words=['let', 'word', 'simply', 'evil'], tags=[4])]
[TaggedDocument(words=['put', 'value', 'male', 'year', 'year', 'old', 'shekel', 'silver', 'place'], tags=[0]), TaggedDocument(words=['man', 'look', 'way'], tags=[1]), TaggedDocument(words=['clearly', 'error', 'sinner', 'self', 'judge'], tags=[2]), TaggedDocument(words=['let', 'wrath', 'overflowing', 'let', 'eye', 'son', 'pride', 'low'], tags=[3]), TaggedDocument(words=['hair', 'cut', 'disease', 'place', 'shut', 'day'], tags=[4])]


Let's Train the Model 

In [ ]:
#It is better to do this in iterations of 4. Think of the Rules of Divisibility 
model = gensim.models.doc2vec.Doc2Vec(vector_size=52, min_count=2, epochs=40)

In [ ]:
model.build_vocab(train_data)

2022-03-31 20:26:30,596 : INFO : collecting all words and their counts
2022-03-31 20:26:30,598 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2022-03-31 20:26:30,649 : INFO : PROGRESS: at example #10000, processed 64032 words (1379341/s), 1792 word types, 10000 tags
2022-03-31 20:26:30,691 : INFO : PROGRESS: at example #20000, processed 127949 words (1566421/s), 2173 word types, 20000 tags
2022-03-31 20:26:30,699 : INFO : collected 2223 word types and 21772 unique tags from a corpus of 21772 examples and 139411 words
2022-03-31 20:26:30,700 : INFO : Loading a fresh vocabulary
2022-03-31 20:26:30,710 : INFO : effective_min_count=2 retains 1594 unique words (71% of original 2223, drops 629)
2022-03-31 20:26:30,711 : INFO : effective_min_count=2 leaves 138782 word corpus (99% of original 139411, drops 629)
2022-03-31 20:26:30,720 : INFO : deleting the raw counts dictionary of 2223 items
2022-03-31 20:26:30,722 : INFO : sample=0.001 downsamples 82 most-comm

In [ ]:
model.train(train_data, total_examples=model.corpus_count, epochs=model.epochs)

2022-03-31 20:26:35,077 : INFO : training model with 3 workers on 1594 vocabulary and 52 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2022-03-31 20:26:36,129 : INFO : EPOCH 1 - PROGRESS: at 28.52% examples, 35854 words/s, in_qsize 5, out_qsize 0
2022-03-31 20:26:37,134 : INFO : EPOCH 1 - PROGRESS: at 71.73% examples, 45381 words/s, in_qsize 4, out_qsize 0
2022-03-31 20:26:37,153 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-03-31 20:26:37,379 : INFO : worker thread finished; awaiting finish of 1 more threads
2022-03-31 20:26:37,392 : INFO : worker thread finished; awaiting finish of 0 more threads
2022-03-31 20:26:37,393 : INFO : EPOCH - 1 : training on 139411 raw words (128921 effective words) took 2.3s, 56159 effective words/s
2022-03-31 20:26:38,409 : INFO : EPOCH 2 - PROGRESS: at 35.73% examples, 46142 words/s, in_qsize 5, out_qsize 0
2022-03-31 20:26:39,382 : INFO : worker thread finished; awaiting finish of 2 more threads
2022-03-31 20:26:3

In [ ]:
#This illustrates the possible vector for the words
print(model.infer_vector(['Jesus', 'is', 'lord']))

[-0.0129045  -0.04868435 -0.01668276  0.06971762 -0.05085002  0.03059414
  0.2589322  -0.01870167 -0.1628906   0.08398718 -0.04720857  0.13495064
  0.21826974  0.18942347  0.16101722 -0.04368481  0.05064658  0.10952795
  0.24997355 -0.11657529  0.21896024  0.12677515  0.07432107 -0.04953406
  0.14903386 -0.20394851 -0.04184566  0.01278181 -0.12460741  0.00891911
  0.01529837 -0.18821745  0.0107184  -0.21841615 -0.03317508  0.10492436
  0.03285783  0.19123328  0.02766942  0.43881527 -0.13668568 -0.05135711
 -0.01935024  0.00552537 -0.16931474  0.04850401 -0.08689576  0.07448645
  0.07529365 -0.01912009  0.02224643  0.10283504]


In [ ]:
model.wv.vocab

{'force': <gensim.models.keyedvectors.Vocab at 0x7ff9253907d0>,
 'wall': <gensim.models.keyedvectors.Vocab at 0x7ff925390990>,
 'town': <gensim.models.keyedvectors.Vocab at 0x7ff925390450>,
 'responsible': <gensim.models.keyedvectors.Vocab at 0x7ff925704210>,
 'land': <gensim.models.keyedvectors.Vocab at 0x7ff925704d50>,
 'let': <gensim.models.keyedvectors.Vocab at 0x7ff925704790>,
 'mind': <gensim.models.keyedvectors.Vocab at 0x7ff925704a50>,
 'sport': <gensim.models.keyedvectors.Vocab at 0x7ff925704b10>,
 'brother': <gensim.models.keyedvectors.Vocab at 0x7ff925704ed0>,
 'comfort': <gensim.models.keyedvectors.Vocab at 0x7ff925704750>,
 'faith': <gensim.models.keyedvectors.Vocab at 0x7ff925704f50>,
 'give': <gensim.models.keyedvectors.Vocab at 0x7ff925704e90>,
 'worship': <gensim.models.keyedvectors.Vocab at 0x7ff925704e10>,
 'news': <gensim.models.keyedvectors.Vocab at 0x7ff9257044d0>,
 'preacher': <gensim.models.keyedvectors.Vocab at 0x7ff925704d90>,
 'word': <gensim.models.keyedvect

How to Find What Text is Most Similar to a Document. 

In [ ]:
from nltk.tokenize import word_tokenize
tokenized_doc = []
for d in data_words:
    tokenized_doc.append(word_tokenize(d.lower()))
tokenized_doc

test_doc = word_tokenize("Jesus is Lord".lower())
model.docvecs.most_similar(positive=[model.infer_vector(test_doc)],topn=5)

2022-03-31 20:28:09,476 : INFO : precomputing L2-norms of doc weight vectors


[(19840, 0.7796519994735718),
 (2894, 0.7564880847930908),
 (20043, 0.7559550404548645),
 (15280, 0.7476166486740112),
 (18709, 0.7164381742477417)]

Ranking the Most Similar Topics 

In [ ]:
#Somehow gensim forgot about dv. If you run it without this line of code, you will get an error. 
model.dv = model.__dict__['docvecs']

In [ ]:
#This may take a while
ranks = []
second_ranks = []
for doc_id in range(len(train_data)):
    inferred_vector = model.infer_vector(train_data[doc_id].words)
    sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))
    rank = [docid for docid, sim in sims].index(doc_id)
    ranks.append(rank)

    second_ranks.append(sims[1])

In [ ]:
import collections

counter = collections.Counter(ranks)
print(counter)
#The most similar document will be the most similar to itself 61.49% of the time.

Counter({0: 19141, 1: 674, 2: 217, 3: 142, 4: 93, 5: 75, 6: 60, 7: 52, 8: 40, 9: 35, 10: 27, 12: 23, 17: 20, 11: 19, 14: 18, 18: 15, 13: 15, 16: 14, 15: 11, 21: 11, 28: 10, 24: 9, 20: 8, 29: 8, 19: 7, 33: 7, 32: 7, 31: 6, 34: 6, 26: 6, 44: 5, 50: 5, 23: 5, 27: 5, 42: 4, 40: 4, 25: 4, 89: 4, 139: 4, 145: 3, 160: 3, 127: 3, 113: 3, 48: 3, 82: 3, 133: 3, 138: 3, 49: 3, 55: 3, 103: 3, 108: 3, 84: 3, 56: 3, 123: 3, 88: 3, 64: 3, 83: 3, 124: 3, 71: 3, 96: 3, 51: 3, 94: 2, 68: 2, 57: 2, 22: 2, 241: 2, 135: 2, 196: 2, 76: 2, 165: 2, 47: 2, 59: 2, 114: 2, 219: 2, 98: 2, 90: 2, 153: 2, 43: 2, 41: 2, 256: 2, 87: 2, 187: 2, 93: 2, 52: 2, 53: 2, 117: 2, 118: 2, 81: 2, 143: 2, 252: 2, 38: 2, 150: 2, 231: 2, 102: 2, 75: 2, 46: 2, 95: 2, 67: 2, 152: 2, 101: 2, 35: 2, 30: 2, 115: 2, 45: 2, 126: 2, 166: 2, 58: 2, 240: 2, 54: 2, 237: 2, 37: 2, 120: 2, 65: 2, 270: 2, 6508: 1, 16867: 1, 2425: 1, 495: 1, 14866: 1, 19251: 1, 19707: 1, 1540: 1, 132: 1, 5834: 1, 9221: 1, 12528: 1, 21372: 1, 21307: 1, 12438: 1,

In [ ]:
#Randomly select a document from the corpus and test similarity and disimilarity 
print('Document ({}): «{}»\n'.format(doc_id, ' '.join(train_data[doc_id].words)))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('SECOND-MOST', 1), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_data[sims[index][0]].words)))

Document (21771): «completely word test breastplate put faith»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d52,n5,w5,mc2,s0.001,t3):

MOST (21771, 0.9173328280448914): «completely word test breastplate put faith»

SECOND-MOST (18428, 0.6178966760635376): «word test safe cover put faith»

MEDIAN (16531, 0.07456423342227936): «write number place put death cross town»

LEAST (8899, -0.4747995138168335): «tribe back part child give»



In [ ]:
# Pick a random document from the corpus and use that document to gather a vector 
import random
doc_id = random.randint(0, len(train_data) - 1)

# Compare and print the second-most-similar document
print('Train Document ({}): «{}»\n'.format(doc_id, ' '.join(train_data[doc_id].words)))
sim_id = second_ranks[doc_id]
print('Similar Document {}: «{}»\n'.format(sim_id, ' '.join(train_data[sim_id[0]].words)))

Train Document (12528): «greatly move»

Similar Document (5533, 0.8430829048156738): «move»



Test Your Model!

In [ ]:
# Pick a random document from the test corpus and infer a vector from the model
doc_id = random.randint(0, len(test_data) - 1)
inferred_vector = model.infer_vector(test_data[doc_id].words)
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))

# Compare and print the most/median/least similar documents from the train corpus
print('Test Document ({}):«{}»\n'.format(doc_id, ''.join(str(test_data[doc_id]))))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)
for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_data[sims[index][0]].words)))

Test Document (4830):«TaggedDocument(['attack', 'mother', 'town', 'hand'], [4830])»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec(dm/m,d52,n5,w5,mc2,s0.001,t3):

MOST (12789, 0.7060545682907104): «brother fear attack put death mother child»

MEDIAN (1998, 0.10183055698871613): «wait baptism wash sin give worship name»

LEAST (6313, -0.47170478105545044): «high place send shake earth nation suddenly move eternal mountain break unchanging hill bent way eternal»



In [ ]:
model.wv.most_similar('fish')

2022-03-31 20:57:27,683 : INFO : precomputing L2-norms of word weight vectors


[('fishing', 0.5958883762359619),
 ('fisherman', 0.5859996676445007),
 ('hook', 0.5516027808189392),
 ('bit', 0.5456740856170654),
 ('net', 0.5281507968902588),
 ('boat', 0.5266973972320557),
 ('basket', 0.5224141478538513),
 ('trading', 0.49856507778167725),
 ('barley', 0.49317318201065063),
 ('planter', 0.48322492837905884)]

Clustering clustering